In [13]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import nibabel as nib
import matplotlib as mpl
%matplotlib inline
import brainsss
import h5py

In [ ]:
# import ants
# import bigbadbrain as bbb
# from scipy.ndimage.filters import gaussian_filter
# from scipy.signal import savgol_filter
# from scipy.interpolate import interp1d

# from skimage.filters import threshold_triangle
# sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages')
# import os
# import statsmodels.api as sm
# import cv2
# import matplotlib.patches as mpatches
# import psutil
# from mpl_toolkits.axes_grid1 import ImageGrid
# from matplotlib.colors import Normalize
# plt.rcParams.update({'font.size': 20})

# from sklearn.feature_extraction.image import grid_to_graph
# from mpl_toolkits.axes_grid1 import ImageGrid
# import json
# from matplotlib.ticker import FuncFormatter
# from sklearn.cluster import AgglomerativeClustering
# from scipy.cluster.hierarchy import dendrogram
# import itertools
# import random
# from scipy.cluster import hierarchy
# from matplotlib.pyplot import cm

# from scipy.interpolate import UnivariateSpline
# from sklearn.linear_model import LinearRegression

# import pickle
# from sklearn.decomposition import PCA
# from sklearn.linear_model import RidgeCV
# from sklearn.linear_model import LassoCV
# from sklearn.linear_model import Ridge
# import pickle

# from scipy.ndimage.morphology import binary_erosion
# from scipy.ndimage.morphology import binary_dilation


In [18]:
import pandas as pd
import pickle

In [19]:
#visual.py from github
def pd_csv_to_h5py(directory, file):
	""" Loads photodiode data from csv file and saves to h5py file.
	Parameters
	----------
	directory: full path to vision folder
	file: csv file
	Returns
	-------
	Nothing. """

	print('loading raw photodiode data... ',end='')
	#load raw data from csv
	load_file = os.path.join(directory, file)
	temp = np.genfromtxt(load_file, delimiter=',',skip_header=1)
	t = temp[:,0]
	ft_triggers = temp[:,1]
	pd1 = temp[:,2]
	pd2 = temp[:,3]
	print('done')

	#save as h5py file
	print('saving photodiode data as h5py file...',end='')
	save_file = os.path.join(directory, 'photodiode.h5')
	with h5py.File(save_file, 'w') as hf:
		hf.create_dataset('time',  data=t)
		hf.create_dataset('ft_triggers',  data=ft_triggers)
		hf.create_dataset('pd1',  data=pd1)
		hf.create_dataset('pd2',  data=pd2)
	print('done')
	
def load_h5py_pd_data(directory):
	""" Loads photodiode data from h5py file.
	Parameters
	----------
	directory: full path to vision folder
	Returns
	-------
	t: 1D numpy array, times of photodiode measurement (in ms)
	pd1: 1D numpy array, photodiode 1 measurements
	pd2: 1D numpy array, photodiode 1 measurements """

	print('loading photodiode data... ',end='')
	#load from h5py file
	load_file = os.path.join(directory, 'photodiode.h5')
	with h5py.File(load_file, 'r') as hf:
		t = hf['time'][:]
		ft_triggers = hf['ft_triggers'][:]
		pd1 = hf['pd1'][:]
		pd2 = hf['pd2'][:]
	print('done')
	return t, ft_triggers, pd1, pd2

def load_photodiode(vision_path):
	""" Tries to load photodiode data from h5py file, and if it doesn't exist loads from csv file.
	Parameters
	----------
	vision_path: full path to vision folder
	Returns
	-------
	t: 1D numpy array, times of photodiode measurement (in ms)
	pd1: 1D numpy array, photodiode 1 measurements
	pd2: 1D numpy array, photodiode 1 measurements """

	# Try to load from h5py file
	try:
		t, ft_triggers, pd1, pd2 = load_h5py_pd_data(vision_path)
		
	# First convert from csv to h5py, then load h5py
	except:
		pd_csv_to_h5py(vision_path,'photodiode.csv')
		t, ft_triggers, pd1, pd2 = load_h5py_pd_data(vision_path)
	return t, ft_triggers, pd1, pd2

def get_stimulus_metadata(vision_path, printlog):
	
	### try to get from pickle ###
	pickle_path = os.path.join(vision_path, 'stimulus_metadata.pkl')
	if os.path.exists(pickle_path):
		printlog("Loaded from Pickle.")
		with open(pickle_path, 'rb') as f:
			metadata = pickle.load(f)
		return metadata['stim_ids'], metadata['angles']
	
	### if no pickle, load from .h5 and save pickle for future ###
	printlog("No pickle; parsing visprotocol .h5")
	fname = [x for x in os.listdir(vision_path) if '.hdf5' in x][0]
	visprotocol_file = os.path.join(vision_path, fname)

	stim_ids = []
	angles = []
	with h5py.File(visprotocol_file, 'r') as f:

		### loop over flies and series to find the one that has many stim presentations (others were aborted)
		# note it is critical each fly has their own .h5 file saved
		fly_ids = list(f['Flies'].keys())
		printlog("Found fly ids: {}".format(fly_ids))
		for fly_id in fly_ids:
			
			series = list(f['Flies'][fly_id]['epoch_runs'].keys())
			printlog("Found series: {}".format(series))
			for serie in series:

				epoch_ids = f['Flies'][fly_id]['epoch_runs'][serie].get('epochs').keys()
				printlog(str(len(epoch_ids)))
				for i, epoch_id in enumerate(epoch_ids):
					stim_id = f['Flies'][fly_id]['epoch_runs'][serie].get('epochs').get(epoch_id).attrs['component_stim_type']
					stim_ids.append(stim_id)
					if stim_id == 'DriftingSquareGrating':
						angle = f['Flies'][fly_id]['epoch_runs'][serie].get('epochs').get(epoch_id).attrs['angle']
						angles.append(angle)
					else:
						angles.append(None)
						
				if len(stim_ids) > 100:
					
					### save pickle for next time
					metadata = {'stim_ids': stim_ids, 'angles': angles}
					save_file = os.path.join(vision_path, 'stimulus_metadata.pkl')
					with open(save_file, 'wb') as f:
						pickle.dump(metadata, f)
					printlog("created {}".format(save_file))
					
					return stim_ids, angles
		printlog('Could not get visual metadata.')
	

def extract_stim_times_from_pd(photodiode_trace, time_vector):
	threshold=0.8,
	command_frame_rate=120
	sample_rate = 10000
	minimum_epoch_separation = 0.9 * (3 + 0) * sample_rate

	# shift & normalize so frame monitor trace lives on [0 1]
	photodiode_trace = photodiode_trace - np.min(photodiode_trace)
	photodiode_trace = photodiode_trace / np.max(photodiode_trace)

	# find frame flip times
	V_orig = photodiode_trace[0:-2]
	V_shift = photodiode_trace[1:-1]
	ups = np.where(np.logical_and(V_orig < threshold, V_shift >= threshold))[0] + 1
	downs = np.where(np.logical_and(V_orig >= threshold, V_shift < threshold))[0] + 1
	frame_times = np.sort(np.append(ups, downs))

	# Use frame flip times to find stimulus start times
	stimulus_start_frames = np.append(0, np.where(np.diff(frame_times) > minimum_epoch_separation)[0] + 1)
	stimulus_end_frames = np.append(np.where(np.diff(frame_times) > minimum_epoch_separation)[0], len(frame_times)-1)
	stimulus_start_times = frame_times[stimulus_start_frames] / sample_rate  # datapoints -> sec
	stimulus_end_times = frame_times[stimulus_end_frames] / sample_rate  # datapoints -> sec

	stim_durations = stimulus_end_times - stimulus_start_times # sec
	return stimulus_start_times

In [14]:
def get_stimulus_metadata(vision_path):

    ### try to get from pickle ###
    pickle_path = os.path.join(vision_path, 'stimulus_metadata.pkl')
    if os.path.exists(pickle_path):
        print("Loaded from Pickle.")
        with open(pickle_path, 'rb') as f:
            metadata = pickle.load(f)
        return metadata['stim_ids'], metadata['angles']

    ### if no pickle, load from .h5 and save pickle for future ###
    print("No pickle; parsing visprotocol .h5")
    fname = [x for x in os.listdir(vision_path) if '.hdf5' in x][0]
    visprotocol_file = os.path.join(vision_path, fname)

    stim_ids = []
    angles = []
    with h5py.File(visprotocol_file, 'r') as f:

        ### loop over flies and series to find the one that has many stim presentations (others were aborted)
        # note it is critical each fly has their own .h5 file saved
        fly_ids = list(f['Flies'].keys())
        print("Found fly ids: {}".format(fly_ids))
        for fly_id in fly_ids:

            series = list(f['Flies'][fly_id]['epoch_runs'].keys())
            print("Found series: {}".format(series))
            for serie in series:

                epoch_ids = f['Flies'][fly_id]['epoch_runs'][serie].get('epochs').keys()
                print(str(len(epoch_ids)))
                for i, epoch_id in enumerate(epoch_ids):
                    stim_id = f['Flies'][fly_id]['epoch_runs'][serie].get('epochs').get(epoch_id).attrs['component_stim_type']
                    stim_ids.append(stim_id)
                    if stim_id == 'DriftingSquareGrating':
                        angle = f['Flies'][fly_id]['epoch_runs'][serie].get('epochs').get(epoch_id).attrs['angle']
                        angles.append(angle)
                    else:
                        angles.append(None)

                if len(stim_ids) > 100:

                    ### save pickle for next time
                    metadata = {'stim_ids': stim_ids, 'angles': angles}
                    save_file = os.path.join(vision_path, 'stimulus_metadata.pkl')
                    with open(save_file, 'wb') as f:
                        pickle.dump(metadata, f)
                    print("created {}".format(save_file))

                    return stim_ids, angles
        print('Could not get visual metadata.')



In [15]:
func_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_116/func_0/'

In [21]:
###########################
### PREP VISUAL STIMULI ###
###########################

vision_path = os.path.join(func_path, 'visual')

### Load Photodiode ###
t, ft_triggers, pd1, pd2 = load_photodiode(vision_path)
stimulus_start_times = extract_stim_times_from_pd(pd2, t)

### Get Metadata ###
stim_ids, angles = get_stimulus_metadata(vision_path)
print(F"Found {len(stim_ids)} presented stimuli.")

# *100 puts in units of 10ms, which will match fictrac
starts_angle_0 = [int(stimulus_start_times[i]*100) for i in range(len(stimulus_start_times)) if angles[i] == 0]
starts_angle_180 = [int(stimulus_start_times[i]*100) for i in range(len(stimulus_start_times)) if angles[i] == 180]
print(F"starts_angle_0: {len(starts_angle_0)}. starts_angle_180: {len(starts_angle_180)}")

loading photodiode data... done


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-1994e37493b6>", line 12, in <module>
    stim_ids, angles = get_stimulus_metadata(vision_path)
TypeError: get_stimulus_metadata() missing 1 required positional argument: 'printlog'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1806, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/IPython/cor

TypeError: get_stimulus_metadata() missing 1 required positional argument: 'printlog'

In [ ]:
np.asarray(starts_angle_0)*10

In [ ]:
list_in_ms = [i*10 for i in starts_angle_0]

In [ ]:
starts_angle_0

In [ ]:
brain_path = os.path.join(func_path, 'functional_channel_2_moco_zscore_highpass.h5')

In [ ]:
timestamps = brainsss.load_timestamps(os.path.join(func_path, 'imaging'), file='functional.xml')

In [ ]:
timestamps.shape

In [ ]:
z=-1
timestamps[-1,0]

In [ ]:
# load slice 20
# interpolate
# make interp object
interp1d
# set stim times to closest datapoint
# loop over all presentations and average

In [ ]:
np.arange(0, 9, 0.1)